In [92]:
import re
from IPython.display import display
import pandas as pd
from unidecode import unidecode
import geonamescache as gnc

rows = {}
lines = set()

# Read all the data, remove diacritics and newlines and store in a set (to remove duplicates)
file = open("data/headlines.txt", 'r')
for line in file.readlines():
    lines.add(unidecode(line).rstrip("\n"))
file.close()

# For each city name, search each headline for that name and store results in a dictionary
cities = [unidecode(city["name"]) for city in gnc.GeonamesCache().get_cities().values()]
for city in cities:
    regexp = re.compile(".*({})( |-|'|\?|,|$)".format(city))
    for line in lines:
        if re.match(regexp, line):
            rows[line]={"city":city}

# For each country name, search each headline for that name and store results in a dictionary
countries = [unidecode(country["name"]) for country in gnc.GeonamesCache().get_countries().values()]
for country in countries:
    regexp = re.compile(".*({})( |-|'|\?|,|$)".format(country))
    for line in lines:
        if re.match(regexp, line):
            row = rows.get(line)
            if row:
                row["country"]=country
            else:
                rows[line]={"country":country}

# move the contents of the dictionary into a DataFrame and display it
data = []
for row in rows.keys():
    data.append([row, rows[row].get("country", "NaN"), rows[row].get("city", "NaN")])
display(pd.DataFrame(data=data,
                  columns=["headline","countries","cities"]).style.hide_index())


headline,countries,cities
Authorities are Worried about the Spread of Norovirus in Dubai,NaN,Dubai
West Nile Virus Symptoms Spread all over Yerevan,NaN,Yerevan
Tuberculosis Hits Luanda,NaN,Luanda
Outbreak of Zika in Santa Elena,NaN,Santa Elena
Will Hepatitis B vaccine help La Paz?,NaN,La Paz
Authorities are Worried about the Spread of Norovirus in Buenos Aires,NaN,Buenos Aires
Hepatitis E re-emerges in Santa Rosa,NaN,Santa Rosa
Bronchitis Symptoms Spread all over Santa Fe,NaN,Santa Fe
Zika spreads to San Luis Potosi,NaN,San Luis
San Juan reports 1st U.S. Zika-related death amid outbreak,NaN,San Juan
